参考资料：
https://www.kaggle.com/ricopue/titanic-the-importance-of-the-family

In [234]:
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')
y = train.Survived

In [235]:
# test['Survived'] = np.nan
# test = test.reindex(columns=list(train.columns))
# sort=False 表示列名按照字母顺序排序以后，再合并，输出的结果自然也是排好序的
data = pd.concat([train, test], axis=0, sort=False)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [236]:
# 连续型变量离散化，男女票价等级作不同处理
def Cal_Fare_Bin(cols):
    Fare = cols[0]
    Sex = cols[1]

    if Sex == 'female':
        if Fare <= 7.5:
            return 'Fare_bin_2'
        elif 7.5 < Fare <= 37:
            return 'Fare_bin_3'
        else:
            return 'Fare_bin_1'

    else:
        if Fare <= 19:
            return 'Fare_bin_3'
        elif 19 < Fare <= 50:
            return 'Fare_bin_2'
        else:
            return 'Fare_bin_1'


data['Fare_bin'] = data[['Fare', 'Sex']].apply(Cal_Fare_Bin, axis=1)

In [237]:
# data['Fare_bin']

In [238]:
# Family_name finally not used is Family_name == Ticked
# data['Family_name'] = data['Name'].str.split(', ').str[0] + '-' + data['Fare'].astype(str)

In [239]:
# 提取出 Title 
data['Title'] = data['Name'].str.split(', ').str[1].str.split('.').str[0]
# data['Title']

In [240]:
data[data['Title']=='Dona']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_bin,Title
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9,C105,C,Fare_bin_1,Dona


In [241]:
data[['Title', 'Survived']].groupby(['Title'], as_index=False).mean().sort_values('Survived')

,Title,Survived
0,Capt,0.000000
2,Don,0.000000
5,Jonkheer,0.000000
15,Rev,0.000000
12,Mr,0.156673
4,Dr,0.428571
1,Col,0.500000
7,Major,0.500000
8,Master,0.575000
9,Miss,0.697802


In [242]:
# 根据 Title 分组
# 级别越高，幸存的概率越大
# Title_bin1 几乎遇难
# Title_bin4 几乎幸存
data['Title'] = data['Title'].replace(['Capt','Don','Jonkheer','Rev','Mr'], 'Title_bin1')
data['Title'] = data['Title'].replace(['Dr','Col','Major','Master'], 'Title_bin2')
data['Title'] = data['Title'].replace(['Miss'], 'Title_bin3')
data['Title'] = data['Title'].replace(['Mrs','Mme','Sir','Ms','Lady','Mlle','the Countess','Dona'], 'Title_bin4')

填充年龄数据的时候，分箱填充。

## 同 transform 方法可以

In [255]:
data.Age.isnull().sum()

0

In [256]:
data.groupby('Pclass')['Age'].mean()

Pclass
1    39.159930
2    29.506705
3    24.816367
Name: Age, dtype: float64

In [257]:
data['Age'] = data.groupby('Pclass')['Age'].transform(
    lambda x: x.fillna(x.mean()))

In [258]:
data.Age.isnull().sum()

0

年龄也分箱处理。

In [259]:
## Age feature do not affect in the same way females or males. Same bins but different age ranges.
def Cal_age_bin(cols):
    Age = cols[0]
    Sex = cols[1]

    # 男女年纪的标准不同
    if Sex == 'female':
        if Age <= 17:
            return 'Age_bin_1'
        elif 18 <= Age <= 31:
            return 'Age_bin_2'
        elif 32 <= Age <= 53:
            return 'Age_bin_3'
        else:
            return 'Age_bin_4'
    else:
        if Age <= 15:
            return 'Age_bin_4'
        elif 16 <= Age <= 24:
            return 'Age_bin_1'
        elif 25 <= Age <= 44:
            return 'Age_bin_3'
        else:
            return 'Age_bin_2'


data['Age_bin'] = data[['Age', 'Sex']].apply(Cal_age_bin, axis=1)

In [260]:
def Cal_Family_bin(cols):
    FamilyZize = cols[0] + cols[1]
    if FamilyZize == 0:
        return 'Alone'
    elif 1 <= FamilyZize <= 3:
        return 'Family'
    elif FamilyZize >= 4:
        return 'Big_family'


data['Family_type'] = data[['SibSp', 'Parch']].apply(Cal_Family_bin, axis=1)

In [261]:
# data['Family_type']

In [262]:
data['Embarked'].isnull().sum()

2

In [263]:
data['Embarked'].mode()

0    S
dtype: object

In [264]:
data.loc[data['Embarked'].isnull(), 'Embarked'] = 'S'

+ 女性 或者 小孩，遇难的
+ 男性 并且 年轻，获救的

In [265]:
# female or child， no survive
data_FC = data[((data['Sex'] == 'female') | (data['Age'] < 14)) & (data['Survived'] == 0)]
# male no child， survive
data_M = data[(data['Sex'] == 'male') & (data['Age'] > 14) & (data['Survived'] == 1)]

In [188]:
# Family relationship got from Ticked and Cabin features
# It can be used also Family_name plus Fare with same results that ticked.
list1 = data_FC['Ticket'].tolist()
list2 = data_FC['Cabin'].tolist()

list3 = data_M['Ticket'].tolist()
list4 = data_M['Cabin'].tolist()

In [269]:
data['Ticket'].isnull().sum()

0

In [270]:
data['Cabin'].isnull().sum()

1014

## 挖掘特征（特别注意）

挖掘特征1：女性或者小孩，死亡的。

挖掘特征2：男性或者青年，存活的。

In [285]:
def FC_dead(row):
    if (row['Ticket'] in list1) or ((str(row['Cabin']) != 'nan') and
                                    (row['Cabin'] in list2)):
        return 1
    else:
        return 0

In [286]:
data['Family_wit_FC_dead'] = data.apply(FC_dead, axis=1)

In [297]:
def M_Alive(row):
    if (row['Ticket'] in list3) or ((str(row['Cabin']) != 'nan') and
                                    (row['Cabin'] in list4)):
        return 1
    else:
        return 0


data['Family_wit_M_alive'] = data.apply(M_Alive, axis=1)

In [288]:
fare_bin = pd.get_dummies(data['Fare_bin'])
title = pd.get_dummies(data['Title'])
sex = pd.get_dummies(data['Sex'], drop_first=True)
age_bin = pd.get_dummies(data['Age_bin'])
family_type = pd.get_dummies(data['Family_type'])
embark = pd.get_dummies(data['Embarked'])
family_wit_FC_dead = data['Family_wit_FC_dead'].astype(np.uint8)
family_with_M_alive = data['Family_wit_M_alive'].astype(np.uint8)

In [300]:
# fare_bin
# title
# sex
# age_bin
# family_type
# embark
# family_wit_FC_dead
# family_with_M_alive

In [301]:
data_cleaned = pd.concat(
    [
        data['Survived'], fare_bin, title, sex, age_bin, family_type, embark,
        family_wit_FC_dead, Family_wit_M_alive
    ],
    axis=1)

In [303]:
data['Survived'].notnull()

0       True
1       True
2       True
3       True
4       True
5       True
6       True
7       True
8       True
9       True
10      True
11      True
12      True
13      True
14      True
15      True
16      True
17      True
18      True
19      True
20      True
21      True
22      True
23      True
24      True
25      True
26      True
27      True
28      True
29      True
       ...  
388    False
389    False
390    False
391    False
392    False
393    False
394    False
395    False
396    False
397    False
398    False
399    False
400    False
401    False
402    False
403    False
404    False
405    False
406    False
407    False
408    False
409    False
410    False
411    False
412    False
413    False
414    False
415    False
416    False
417    False
Name: Survived, Length: 1309, dtype: bool

In [307]:
train_cleaned = data_cleaned[data['Survived'].notnull()]
test_cleaned = data_cleaned[data['Survived'].isnull()]

In [310]:
train_cleaned.shape

(891, 21)

In [311]:
test_cleaned.shape

(418, 21)

In [312]:
del test_cleaned['Survived']

In [314]:
X = train_cleaned.drop('Survived', axis=1)

In [317]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=1)

### 逻辑回归

In [322]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='lbfgs')
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [324]:
y_pred = lr.predict(X_test)

In [331]:
y_test.value_counts()

0    128
1     95
Name: Survived, dtype: int64

In [333]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=['遇难', '幸存']))

              precision    recall  f1-score   support

          遇难       0.98      0.98      0.98       128
          幸存       0.97      0.97      0.97        95

   micro avg       0.97      0.97      0.97       223
   macro avg       0.97      0.97      0.97       223
weighted avg       0.97      0.97      0.97       223



In [337]:
from sklearn.model_selection import cross_val_score

lr_scores = cross_val_score(lr, X, y, cv=5, verbose=2)
lr_scores.mean()

[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished


0.9809362551143046

### 集成学习 XGBClassifier

In [334]:
from xgboost import XGBClassifier

xgbc = XGBClassifier()
xgbc.fit(X_train, y_train)
xgbc.score(X_test, y_test)

0.9820627802690582

In [203]:
y_pred = xgbc.predict(test_cleaned)
res = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_pred})
res.to_csv('xgbc-submission.csv', index=False)